In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('limited_dataset.csv')
df.head()

/var/folders/_d/82p978c554zg2gg3cn3vrbq00000gn/T/ipykernel_4102/501569260.py:1: DtypeWarning: Columns (3,4,5,16,17,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('limited_dataset.csv')


,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,...,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59
0,Gen Hoshino,Comedy,Comedy,73,230666,FALSE,0.676,0.461,1.0,-6.746,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Gen Hoshino,Comedy,Comedy,73,230666,FALSE,0.676,0.461,1.0,-6.746,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Gen Hoshino,Comedy,Comedy,73,230666,FALSE,0.676,0.461,1.0,-6.746,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Gen Hoshino,Comedy,Comedy,73,230666,FALSE,0.676,0.461,1.0,-6.746,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Gen Hoshino,Comedy,Comedy,73,230666,FALSE,0.676,0.461,1.0,-6.746,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110000 entries, 0 to 109999
Data columns (total 60 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   artists           110000 non-null  object 
 1   album_name        110000 non-null  object 
 2   track_name        110000 non-null  object 
 3   popularity        110000 non-null  object 
 4   duration_ms       110000 non-null  object 
 5   explicit          110000 non-null  object 
 6   danceability      110000 non-null  float64
 7   energy            110000 non-null  float64
 8   key               110000 non-null  float64
 9   loudness          110000 non-null  float64
 10  mode              110000 non-null  float64
 11  speechiness       110000 non-null  float64
 12  acousticness      110000 non-null  float64
 13  instrumentalness  110000 non-null  float64
 14  liveness          110000 non-null  float64
 15  valence           110000 non-null  float64
 16  tempo             11

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from scipy.sparse import coo_matrix
from implicit.als import AlternatingLeastSquares
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from math import sqrt

# Load dataset
df = pd.read_csv('limited_dataset.csv')

df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')
df = df.dropna(subset=['date_added'])

# Remove last 3 songs from each playlist
data_sorted = df.sort_values(by=['playlist_name', 'date_added'])
train_data, removed_songs = [], []

for playlist, group in data_sorted.groupby('playlist_name'):
    if len(group) > 3:
        removed_songs.append(group.tail(3))
        train_data.append(group.head(len(group) - 3))

train_data = pd.concat(train_data)
removed_songs = pd.concat(removed_songs)

# Encode categorical variables
track_encoder = LabelEncoder()
df["encoded_track"] = track_encoder.fit_transform(df["track_name"])
playlist_encoder = LabelEncoder()
df["encoded_playlist"] = playlist_encoder.fit_transform(df["playlist_name"])

# Create interaction matrix
interaction_matrix = coo_matrix((np.ones(len(df)), (df['encoded_playlist'], df['encoded_track'])))

# Train ALS model
als_model = AlternatingLeastSquares(factors=50, regularization=0.1, iterations=20)
als_model.fit(interaction_matrix)

# Function to recommend songs
def recommend_songs(input_tracks, top_n=3):
    input_tracks_encoded = [track_encoder.transform([t])[0] for t in input_tracks if t in track_encoder.classes_]
    if not input_tracks_encoded:
        return []
    
    recommendations = set()
    for track in input_tracks_encoded:
        recs = als_model.similar_items(track, N=top_n+1)[1:]
        recommendations.update([track_encoder.inverse_transform([r[0]])[0] for r in recs])
    
    return list(recommendations)[:top_n]

# Create song similarity matrix
num_features = ["danceability", "energy", "valence", "tempo", "loudness", "speechiness", "acousticness", "instrumentalness"]
scaler = MinMaxScaler()
df[num_features] = scaler.fit_transform(df[num_features])

feature_vectors = df[num_features].values
song_similarity_matrix = cosine_similarity(feature_vectors)
song_similarity_df = pd.DataFrame(song_similarity_matrix, index=df['track_name'], columns=df['track_name'])

# Function to compute similarity
def compute_similarity(song1, song2, song_similarity_df):
    if song1 in song_similarity_df.index and song2 in song_similarity_df.columns:
        return song_similarity_df.loc[song1, song2]
    return 0

# Function to calculate RMSE
def calculate_rmse(recommended_songs, removed_songs, song_similarity_df):
    all_errors = []
    for _, removed_group in removed_songs.groupby('playlist_name'):
        playlist_songs = removed_group['track_name'].tolist()
        for song in playlist_songs:
            recommended_song = recommended_songs.get(song, None)
            if recommended_song:
                distances = [compute_similarity(song, recommended, song_similarity_df) for recommended in recommended_song]
                all_errors.append(min(distances))
    return sqrt(mean_squared_error(np.ones(len(all_errors)), all_errors))

# Generate recommendations for first 100 playlists
all_recommended_songs = {}
for playlist, group in list(data_sorted.groupby('playlist_name'))[:100]:
    playlist_songs = group['track_name'].tolist()
    all_recommended_songs[playlist] = recommend_songs(playlist_songs, top_n=1)

# Calculate RMSE
rmse_value = calculate_rmse(all_recommended_songs, removed_songs, song_similarity_df)
print(f"Root Mean Squared Error (RMSE): {rmse_value}")


/var/folders/_d/82p978c554zg2gg3cn3vrbq00000gn/T/ipykernel_4102/3677905146.py:11: DtypeWarning: Columns (3,4,5,16,17,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('limited_dataset.csv')
/Users/arkdawg/miniforge3/envs/dsc80/lib/python3.12/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
/Users/arkdawg/miniforge3/envs/dsc80/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.002482891082763672 seconds
  warnings.warn(


  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def random_playlist_recommendations(num_samples=5, top_n=3):
    unique_playlists = df["playlist_name"].unique()
    recommendations = []

    for _ in range(num_samples):
        # Randomly select a playlist
        playlist_name = random.choice(unique_playlists)
        # Get songs from the selected playlist
        playlist_songs = df[df["playlist_name"] == playlist_name]["track_id"].unique()

        # Randomly select some songs from the playlist as input
        input_tracks = random.sample(list(playlist_songs), min(3, len(playlist_songs)))  # Choose up to 3 songs

        # Get recommendations
        recommended_songs = recommend_songs(input_tracks, top_n)
        recommendations.append((playlist_name, input_tracks, recommended_songs))

    return recommendations


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def evaluate_model():
    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        df[["track_id", "playlist_name"]],
        np.ones(len(df)),  # Assuming all songs in playlists are positive
        test_size=0.7,  # Use 20% of the data for testing
        random_state=42
    )
    
    # Train the model on the training set
    valid_train_mask = X_train["track_id"].isin(track_encoder.classes_) & X_train["playlist_name"].isin(playlist_encoder.classes_)
    X_train_valid = X_train[valid_train_mask]

    # Ensure valid data
    if X_train_valid.empty:
        print("No valid training data found after filtering.")
        return

    # Encode training data
    X_train_encoded = [
        track_encoder.transform(X_train_valid["track_id"]),
        playlist_encoder.transform(X_train_valid["playlist_name"])
    ]
    
    # Get corresponding y_train
    y_train_valid = y_train[valid_train_mask]

    # Ensure sizes match
    if len(X_train_encoded[0]) != len(y_train_valid):
        print(f"Size mismatch: X_train_encoded: {len(X_train_encoded[0])}, y_train_valid: {len(y_train_valid)}")
        return

    model.fit(X_train_encoded, y_train_valid, epochs=10, batch_size=64)

    # Test the model on the test set
    valid_test_mask = X_test["track_id"].isin(track_encoder.classes_) & X_test["playlist_name"].isin(playlist_encoder.classes_)
    X_test_valid = X_test[valid_test_mask]
    
    if X_test_valid.empty:
        print("No valid test data found after filtering.")
        return

    # Encode test data
    X_test_encoded = [
        track_encoder.transform(X_test_valid["track_id"]),
        playlist_encoder.transform(X_test_valid["playlist_name"])
    ]
    
    # Get corresponding y_test
    y_test_valid = y_test[valid_test_mask]
    
    if len(X_test_encoded[0]) != len(y_test_valid):
        print(f"Size mismatch: X_test_encoded: {len(X_test_encoded[0])}, y_test_valid: {len(y_test_valid)}")
        return

    # Predictions
    y_pred = model.predict(X_test_encoded)
    y_pred_classes = (y_pred > 0.5).astype(int)  # Convert probabilities to binary predictions

    # Calculate accuracy
    accuracy = accuracy_score(y_test_valid, y_pred_classes)
    print(f"Model Accuracy: {accuracy:.4f}")


# Call the evaluate function
evaluate_model()



In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.optimizers import Adam

# df = pd.read_csv('subset_combined_dataset.csv')
df = pd.read_csv('limited_dataset.csv')


# Convert track_id to string
df['track_name'] = df['track_id'].astype(str)

# Encode categorical variables
track_encoder = LabelEncoder()
df["encoded_track_id"] = track_encoder.fit_transform(df["track_id"])

playlist_encoder = LabelEncoder()
df["playlist_name"] = playlist_encoder.fit_transform(df["playlist_name"])

# Normalize numeric features
num_features = ["danceability", "energy", "valence", "tempo", "loudness", "speechiness", "acousticness", "instrumentalness"]
scaler = MinMaxScaler()
df[num_features] = scaler.fit_transform(df[num_features])

# Define model hyperparameters
embedding_dim = 32
n_tracks = df["encoded_track_id"].nunique()
n_playlists = df["playlist_name"].nunique()

# Define inputs
track_input = Input(shape=(1,))
playlist_input = Input(shape=(1,))

# Embedding layers
track_embedding = Embedding(n_tracks, embedding_dim)(track_input)
playlist_embedding = Embedding(n_playlists, embedding_dim)(playlist_input)

# Flatten embeddings
track_vec = Flatten()(track_embedding)
playlist_vec = Flatten()(playlist_embedding)

# Compute interaction
dot_product = Dot(axes=1)([track_vec, playlist_vec])

# Additional dense layers for learning non-linear relationships
concat = Concatenate()([dot_product, track_vec, playlist_vec])
dense1 = Dense(64, activation="relu")(concat)
dense2 = Dense(32, activation="relu")(dense1)
output = Dense(1, activation="sigmoid")(dense2)

# Compile model
model = Model(inputs=[track_input, playlist_input], outputs=output)
model.compile(optimizer=Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=["accuracy"])

# Prepare training data
X_train = [df["encoded_track_id"].values, df["playlist_name"].values]
y_train = np.ones(len(df))  # Implicit feedback (all songs in playlists are positive)

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64)

def recommend_songs(input_tracks, top_n=3):
    # Ensure input_tracks are converted to strings for consistency
    input_tracks = [str(track) for track in input_tracks]
    input_tracks_encoded = track_encoder.transform([t for t in input_tracks if t in track_encoder.classes_])
    
    if len(input_tracks_encoded) == 0:
        print("No valid input tracks found in the dataset.")
        return []
    
    playlist_ids = np.arange(n_playlists)
    
    scores = model.predict([
        np.repeat(input_tracks_encoded, len(playlist_ids)), 
        np.tile(playlist_ids, len(input_tracks_encoded))
    ])
    
    avg_scores = np.mean(scores, axis=0)  # Average predictions for multiple input songs
    top_indices = np.argsort(avg_scores)[-top_n:][::-1]  # Get top-N recommendations
    
    # Ensure recommended indices exist in track_encoder
    valid_indices = [idx for idx in top_indices if idx < len(track_encoder.classes_)]
    
    if not valid_indices:
        print("No valid recommendations found.")
        return []
    
    # To ensure uniqueness, you can convert the result to a set and back to a list
    recommended_tracks = list(set(track_encoder.inverse_transform(valid_indices)))
    
    return recommended_tracks[:top_n]  # Return the top N unique recommendations

def get_song_name(recommend_song_ids):
    # Ensure the input is a list of strings
    if isinstance(recommend_song_ids, np.ndarray):
        recommend_song_ids = recommend_song_ids.tolist()
    elif not isinstance(recommend_song_ids, list):
        recommend_song_ids = [recommend_song_ids]  # Convert a single ID to a list

    # Convert all track IDs to string (to match df["track_id"])
    recommend_song_ids = [str(track_id) for track_id in recommend_song_ids]

    # Filter DataFrame for matching track IDs
    recommended_songs = df[df["track_id"].isin(recommend_song_ids)][["track_id", "track_name"]].drop_duplicates()

    return recommended_songs["track_name"].tolist()

# Example usage
input_tracks = ["6lfxq3CG4xtTiEg7opyCyx", "1EzrEOXmMH3G43AXT1y7pA", "5ivF4eQBqJiVL5IAE9jRyl"]  # Replace with actual track IDs
recommended_songs = recommend_songs(input_tracks)
print_names = get_song_name(recommended_songs)
print("Recommended Songs:", print_names)



In [ ]:
from sklearn.metrics import mean_squared_error

def evaluate_model(df, model, track_encoder, playlist_encoder, num_features):
    playlist_mse = []
    
    for playlist, group in df.groupby("playlist_name"):
        if len(group) < 4:
            continue  # Skip short playlists
        
        train_tracks = group.iloc[:-3]  # All but the last 3 tracks
        test_tracks = group.iloc[-3:]  # Last 3 tracks
        
        playlist_encoded = playlist_encoder.transform([playlist])[0]
        train_track_ids = train_tracks["encoded_track_id"].values
        test_features = test_tracks[num_features].values  # Numerical features of excluded songs
        
        # Predict next track
        predictions = model.predict([
            np.tile(train_track_ids, len(track_encoder.classes_)),
            np.repeat(playlist_encoded, len(track_encoder.classes_))
        ])
        
        predicted_track_idx = np.argmax(predictions)  # Track with highest predicted score
        predicted_track_id = track_encoder.inverse_transform([predicted_track_idx])[0]
        
        # Retrieve numerical features of predicted track
        predicted_features = df[df["track_id"] == predicted_track_id][num_features].values
        if len(predicted_features) == 0:
            continue  # Skip if the predicted track is missing from the dataset
        
        # Compute MSE against test tracks
        mse = min(mean_squared_error(test_features[i], predicted_features[0]) for i in range(len(test_features)))
        playlist_mse.append(mse)
    
    return np.mean(playlist_mse) if playlist_mse else None

# Evaluate model
mse_score = evaluate_model(df, model, track_encoder, playlist_encoder, num_features)
print("Mean MSE across playlists:", mse_score)


In [ ]:
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def random_playlist_recommendations(num_samples=5, top_n=3):
    unique_playlists = df["playlist_name"].unique()
    recommendations = []

    for _ in range(num_samples):
        # Randomly select a playlist
        playlist_name = random.choice(unique_playlists)
        # Get songs from the selected playlist
        playlist_songs = df[df["playlist_name"] == playlist_name]["track_id"].unique()

        # Randomly select some songs from the playlist as input
        input_tracks = random.sample(list(playlist_songs), min(3, len(playlist_songs)))  # Choose up to 3 songs

        # Get recommendations
        recommended_songs = recommend_songs(input_tracks, top_n)
        recommendations.append((playlist_name, input_tracks, recommended_songs))

    return recommendations


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def evaluate_model():
    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        df[["track_id", "playlist_name"]],
        np.ones(len(df)),  # Assuming all songs in playlists are positive
        test_size=0.7,  # Use 20% of the data for testing
        random_state=42
    )
    
    # Train the model on the training set
    valid_train_mask = X_train["track_id"].isin(track_encoder.classes_) & X_train["playlist_name"].isin(playlist_encoder.classes_)
    X_train_valid = X_train[valid_train_mask]

    # Ensure valid data
    if X_train_valid.empty:
        print("No valid training data found after filtering.")
        return

    # Encode training data
    X_train_encoded = [
        track_encoder.transform(X_train_valid["track_id"]),
        playlist_encoder.transform(X_train_valid["playlist_name"])
    ]
    
    # Get corresponding y_train
    y_train_valid = y_train[valid_train_mask]

    # Ensure sizes match
    if len(X_train_encoded[0]) != len(y_train_valid):
        print(f"Size mismatch: X_train_encoded: {len(X_train_encoded[0])}, y_train_valid: {len(y_train_valid)}")
        return

    model.fit(X_train_encoded, y_train_valid, epochs=10, batch_size=64)

    # Test the model on the test set
    valid_test_mask = X_test["track_id"].isin(track_encoder.classes_) & X_test["playlist_name"].isin(playlist_encoder.classes_)
    X_test_valid = X_test[valid_test_mask]
    
    if X_test_valid.empty:
        print("No valid test data found after filtering.")
        return

    # Encode test data
    X_test_encoded = [
        track_encoder.transform(X_test_valid["track_id"]),
        playlist_encoder.transform(X_test_valid["playlist_name"])
    ]
    
    # Get corresponding y_test
    y_test_valid = y_test[valid_test_mask]
    
    if len(X_test_encoded[0]) != len(y_test_valid):
        print(f"Size mismatch: X_test_encoded: {len(X_test_encoded[0])}, y_test_valid: {len(y_test_valid)}")
        return

    # Predictions
    y_pred = model.predict(X_test_encoded)
    y_pred_classes = (y_pred > 0.5).astype(int)  # Convert probabilities to binary predictions

    # Calculate accuracy
    accuracy = accuracy_score(y_test_valid, y_pred_classes)
    print(f"Model Accuracy: {accuracy:.4f}")


# Call the evaluate function
evaluate_model()



In [ ]:
# Generate random playlist recommendations
recommendations = random_playlist_recommendations(num_samples=5, top_n=3)
for playlist_name, input_tracks, recommended_songs in recommendations:
    print(f"Playlist: {playlist_name}, Input Tracks: {input_tracks}, Recommended Songs: {recommended_songs}")

# Evaluate model performance
evaluate_model()


In [ ]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy
import pandas as pd

# Load dataset
df = pd.read_csv('cleaned_dataset_v2.csv')

# Define a Reader object
reader = Reader(rating_scale=(0, 1))

# Create a Surprise dataset from the DataFrame
data = Dataset.load_from_df(df[['playlist_name', 'track_id', 'rating']], reader)

# Function to evaluate model performance
def evaluate_model(playlist_name):
    # Filter the playlist data
    playlist_data = df[df['playlist_name'] == playlist_name]
    
    # Check if the playlist has more than 3 songs
    if len(playlist_data) <= 3:
        print("Playlist must have more than 3 songs.")
        return None
    
    # Separate the last three songs for prediction
    hidden_songs = playlist_data['track_id'].values[-3:]
    train_data = playlist_data.iloc[:-3]
    
    # Create a new dataset for training
    trainset = Dataset.load_from_df(train_data[['playlist_name', 'track_id', 'rating']], reader).build_full_trainset()

    # User-based collaborative filtering model
    model = KNNBasic(sim_options={'user_based': True})
    model.fit(trainset)

    # Prepare test set for prediction
    testset = [(trainset.to_inner_uid(train_data['playlist_name'].iloc[0]), 
                 trainset.to_inner_iid(track_id)) for track_id in hidden_songs]
    
    # Make predictions
    predictions = model.test(testset)
    
    # Evaluate the predictions
    predicted_tracks = [track_encoder.inverse_transform([pred[1]])[0] for pred in predictions if pred.est >= 0.5]
    
    return predicted_tracks, hidden_songs.tolist()

# Example usage
playlist_name = "example_playlist"  # Replace with actual playlist name
predicted_songs, actual_songs = evaluate_model(playlist_name)
print("Predicted Songs:", predicted_songs)
print("Actual Songs:", actual_songs)
